### Get HIBP Data 

1. For everypol
2. For all breaches
3. For all bihar pols

In [1]:
import logging
import openpyxl
import os
import time
import json
from typing import Dict, Set
import requests
import pandas as pd
from pathlib import Path

In [2]:
with open("hibp_key", "r") as key_file:
    hibp_api_key = key_file.read().strip()

payload = {}
headers = {
    'hibp-api-key': hibp_api_key,
    'format': 'application/json',
    'timeout': '2.5',
    'HIBP': hibp_api_key,
    'user-agent': 'PythonScript'
}

In [3]:
# Rate limit
rate_limit_interval = 6
last_request_time = time.time()

In [4]:
class HIBPProcessor:
    def __init__(self, output_folder: str, headers: Dict, payload: Dict, rate_limit_interval: float):
        self.output_folder = Path(output_folder)
        self.headers = headers
        self.payload = payload
        self.rate_limit_interval = rate_limit_interval
        self.processed_emails: Set[str] = set()
        self.last_request_time = 0
        
        # Ensure output directory exists
        self.output_folder.mkdir(parents=True, exist_ok=True)
        
        # Set up logging
        self._setup_logging()
        
    def _setup_logging(self):
        """Configure logging with both file and console handlers."""
        log_format = '%(asctime)s - %(levelname)s - %(message)s'
        logging.basicConfig(
            level=logging.INFO,
            format=log_format,
            handlers=[
                logging.FileHandler(self.output_folder / 'processing.log'),
                logging.StreamHandler()
            ]
        )
    
    def _load_processed_emails(self) -> Set[str]:
        """Load previously processed emails from existing files and logs."""
        processed = set()
        
        # Check existing JSON files
        for file_path in self.output_folder.glob('*.json'):
            processed.add(file_path.stem)
            
        # Check log file for processed entries
        log_path = self.output_folder / 'processing.log'
        if log_path.exists():
            with log_path.open('r') as f:
                for line in f:
                    if "Successfully processed" in line:
                        # Extract email from log line using string manipulation
                        # Assumes format: "[X/Y] email@domain.com - Successfully processed"
                        try:
                            email = line.split(']')[1].split('-')[0].strip()
                            processed.add(email)
                        except IndexError:
                            continue
                            
        return processed
    
    def _wait_for_rate_limit(self):
        """Enforce rate limiting between requests."""
        now = time.time()
        elapsed_time = now - self.last_request_time
        if elapsed_time < self.rate_limit_interval:
            time.sleep(self.rate_limit_interval - elapsed_time)
        self.last_request_time = time.time()
    
    def _query_hibp_api(self, email: str) -> tuple[bool, dict]:
        """
        Query the HIBP API for a single email.
        Returns (success, response_data) tuple.
        """
        url = f"https://haveibeenpwned.com/api/v3/breachedaccount/{email}"
        response = None
        
        try:
            self._wait_for_rate_limit()
            response = requests.get(url, headers=self.headers, data=self.payload)
            
            if response.status_code == 404:
                return True, {}
                
            if response.status_code == 429:
                retry_after = int(response.headers.get("Retry-After", 1))
                logging.warning(f"Rate limit hit. Waiting {retry_after} seconds")
                time.sleep(retry_after + 2)
                return False, {}
                
            response.raise_for_status()
            return True, response.json()
            
        except requests.exceptions.RequestException as e:
            status_code = getattr(response, 'status_code', 'N/A')
            logging.error(f"Request failed: {e} (Status Code: {status_code})")
            return False, {}
        except json.JSONDecodeError as e:
            logging.error(f"Invalid JSON response: {e}")
            return False, {}
        except Exception as e:
            logging.error(f"Unexpected error: {e}")
            return False, {}
    
    def process_dataframe(self, df: pd.DataFrame) -> Dict:
        """
        Process all emails in the DataFrame.
        Returns statistics about the processing run.
        """
        if 'email' not in df.columns:
            raise ValueError("DataFrame must contain an 'email' column")
            
        # Load previously processed emails
        self.processed_emails = self._load_processed_emails()
        
        total_emails = len(df)
        stats = {
            'total': total_emails,
            'processed': 0,
            'skipped': 0,
            'errors': 0
        }
        
        logging.info(f"Starting to process {total_emails} emails")
        
        for index, row in df.iterrows():
            email = row['email']
            log_prefix = f"[{index + 1}/{total_emails}] {email}"
            
            # Skip if already processed
            if email in self.processed_emails:
                logging.info(f"{log_prefix} - Skipping (previously processed)")
                stats['skipped'] += 1
                continue
                
            # Query API and handle response
            success, response_data = self._query_hibp_api(email)
            
            if not success:
                stats['errors'] += 1
                continue
                
            # Save response data (even if empty for no breaches)
            output_path = self.output_folder / f"{email}.json"
            try:
                with output_path.open('w') as f:
                    json.dump(response_data, f, indent=2)
                logging.info(f"{log_prefix} - Successfully processed and saved")
                self.processed_emails.add(email)
                stats['processed'] += 1
            except IOError as e:
                logging.error(f"{log_prefix} - Failed to save results: {e}")
                stats['errors'] += 1
        
        # Log final statistics
        logging.info(f"""
        Processing completed:
        - Total emails: {stats['total']}
        - Processed: {stats['processed']}
        - Skipped: {stats['skipped']}
        - Errors: {stats['errors']}
        """)
        
        return stats

In [5]:
def clean_email_column(df, column_name="email"):
    """
    Cleans the specified email column in a DataFrame by:
    1. Stripping whitespace, converting to lowercase, and removing commas.
    2. Dropping rows where the email contains only a single letter or symbol.
    3. Dropping rows where the email is NaN.
    4. Drop dupes
    5. Valid email

    Args:
        df (pd.DataFrame): The DataFrame to clean.
        column_name (str): The column to process (default: "email").

    Returns:
        pd.DataFrame: Cleaned DataFrame (modification done safely).
    """
    if column_name in df.columns:
        df = df.copy()
        df[column_name] = df[column_name].str.strip().str.lower().str.replace(",", "", regex=True).str.replace(" ", "")
        df = df[~df[column_name].str.match(r"^[A-Za-z,_-]$", na=False)]
        df = df.dropna(subset=[column_name])
        df = df.drop_duplicates(subset=[column_name], keep="first")
        
        email_regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'

        df = df[df[column_name].str.match(email_regex, na=False)]

    return df

In [6]:
output_folder = "../data/everypol/everypol_hibp/"
processor = HIBPProcessor(output_folder, headers, payload, 10)
everypol_df = pd.read_csv("../data/everypol/everypol_unique_emails.csv")
everypol_df = clean_email_column(everypol_df)
stats = processor.process_dataframe(everypol_df)
print(f"Processing complete. Processed {stats['processed']} emails with {stats['errors']} errors.")

2025-02-01 12:26:24,118 - INFO - Starting to process 8512 emails
2025-02-01 12:26:24,119 - INFO - [1/8512] shivajirao@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,120 - INFO - [2/8512] adhikari.deepak@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,120 - INFO - [3/8512] sisiradhikari76@yahoo.com - Skipping (previously processed)
2025-02-01 12:26:24,120 - INFO - [4/8512] adhikari.suvendu@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,121 - INFO - [5/8512] yogi.adityanath@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,121 - INFO - [6/8512] av.adsul@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,122 - INFO - [7/8512] officelka@gmail.com - Skipping (previously processed)
2025-02-01 12:26:24,122 - INFO - [8/8512] rajendra.agrawal51@gmail.com - Skipping (previously processed)
2025-02-01 12:26:24,123 - INFO - [9/8512] eahmed@hotmail.com - Skipping (previously processed)
2025-02-01 12:26:2

2025-02-01 12:26:24,166 - INFO - [81/8512] ramtahal.choudhary@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,166 - INFO - [82/8512] ahk.chowdhury@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,167 - INFO - [83/8512] adhir@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,167 - INFO - [84/8512] chudasamar.naranbhai@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,168 - INFO - [85/8512] col.sonaram@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,168 - INFO - [86/8512] raosaheb.danve@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,169 - INFO - [87/8512] kakoli.dastidar@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,170 - INFO - [88/8512] sankar.datta@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,170 - INFO - [89/8512] bandaru@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,171 - INFO - [90/8512] ratna.de@sansa

2025-02-01 12:26:24,210 - INFO - [161/8512] joshi.pralhad@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,210 - INFO - [162/8512] niranajan.jyoti@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,211 - INFO - [163/8512] parasuraman.k@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,211 - INFO - [164/8512] mpamreli@gmail.com - Skipping (previously processed)
2025-02-01 12:26:24,212 - INFO - [165/8512] mahboobali.kaiser@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,212 - INFO - [166/8512] kalvakuntla.kavitha@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,213 - INFO - [167/8512] knshikarpur@gmail.com - Skipping (previously processed)
2025-02-01 12:26:24,213 - INFO - [168/8512] k.kamaraj@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,214 - INFO - [169/8512] k.haribabu@gmail.com - Skipping (previously processed)
2025-02-01 12:26:24,214 - INFO - [170/8512] bjpdpkpl@gmail.c

2025-02-01 12:26:24,266 - INFO - [239/8512] daddan.mishra@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,267 - INFO - [240/8512] kalraj.mishra@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,267 - INFO - [241/8512] ajay.kumar19@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,268 - INFO - [242/8512] pinakimisra@hotmail.com - Skipping (previously processed)
2025-02-01 12:26:24,268 - INFO - [243/8512] appt.pmo@gov.in - Skipping (previously processed)
2025-02-01 12:26:24,269 - INFO - [244/8512] masrarul.haque@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,270 - INFO - [245/8512] mohammed.faizal@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,270 - INFO - [246/8512] pc.mohan@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,270 - INFO - [247/8512] sssm234@gmail.com - Skipping (previously processed)
2025-02-01 12:26:24,271 - INFO - [248/8512] vmoily@kar.nic.in - Skipping (p

2025-02-01 12:26:24,311 - INFO - [318/8512] krishna.raj19@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,311 - INFO - [319/8512] harinarayan.rajbhar@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,312 - INFO - [320/8512] mb.rajesh@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,312 - INFO - [321/8512] dr.manojrajoria001@gmail.com - Skipping (previously processed)
2025-02-01 12:26:24,312 - INFO - [322/8512] mukesh.rajput@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,313 - INFO - [323/8512] goka.raju@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,313 - INFO - [324/8512] ram.charitra@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,314 - INFO - [325/8512] rammohannaidu.k@sansad.nic.in - Skipping (previously processed)
2025-02-01 12:26:24,314 - INFO - [326/8512] office.mpagra@gmail.com - Skipping (previously processed)
2025-02-01 12:26:24,315 - INFO - [328/8512] vishnu

...

2025-02-01 12:28:28,543 - INFO - [6731/8512] jan_kecskes@nrsr.sk - Skipping (previously processed)
2025-02-01 12:28:28,545 - INFO - [6732/8512] jan_marosz@nrsr.sk - Skipping (previously processed)
2025-02-01 12:28:28,546 - INFO - [6733/8512] jan_mora@nrsr.sk - Skipping (previously processed)
2025-02-01 12:28:28,547 - INFO - [6734/8512] karol_farkasovsky@nrsr.sk - Skipping (previously processed)
2025-02-01 12:28:28,548 - INFO - [6735/8512] karol_galek@nrsr.sk - Skipping (previously processed)
2025-02-01 12:28:28,550 - INFO - [6736/8512] katarina_csefalvayova@nrsr.sk - Skipping (previously processed)
2025-02-01 12:28:28,550 - INFO - [6737/8512] katarina_machackova@nrsr.sk - Skipping (previously processed)
2025-02-01 12:28:28,551 - INFO - [6738/8512] ladislav_balodi@nrsr.sk - Skipping (previously processed)
2025-02-01 12:28:28,551 - INFO - [6739/8512] magdalena_kucianova@nrsr.sk - Skipping (previously processed)
2025-02-01 12:28:28,553 - INFO - [6740/8512] marek_krajci@nrsr.sk - Skipping 

2025-02-01 12:28:36,498 - INFO - [8375/8512] christine.grahame.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,498 - INFO - [8376/8512] claire.baker.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,499 - INFO - [8377/8512] clare.adamson.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,500 - INFO - [8378/8512] clare.haughey.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,500 - INFO - [8379/8512] claudia.beamish.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,501 - INFO - [8380/8512] colin.beattie.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,501 - INFO - [8381/8512] colin.smyth.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,502 - INFO - [8382/8512] daniel.johnson.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,502 - INFO - [8383/8512] david.stewart.msp@parliament.scot - Skipping (p

2025-02-01 12:28:36,531 - INFO - [8448/8512] mike.rumbles.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,531 - INFO - [8449/8512] miles.briggs.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,532 - INFO - [8450/8512] monica.lennon.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,532 - INFO - [8451/8512] murdo.fraser.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,532 - INFO - [8452/8512] neil.bibby.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,533 - INFO - [8453/8512] neil.findlay.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,533 - INFO - [8454/8512] nicola.sturgeon.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,534 - INFO - [8455/8512] oliver.mundell.msp@parliament.scot - Skipping (previously processed)
2025-02-01 12:28:36,534 - INFO - [8456/8512] patrick.harvie.msp@parliament.scot - Skipping (previous

2025-02-01 12:28:46,484 - INFO - [8523/8512] mark.reckless@assembly.wales - Skipping (previously processed)
2025-02-01 12:28:46,485 - INFO - [8524/8512] michelle.brown@assembly.wales - Skipping (previously processed)
2025-02-01 12:28:46,486 - INFO - [8525/8512] mick.antoniw@assembly.wales - Skipping (previously processed)
2025-02-01 12:28:46,487 - INFO - [8526/8512] mike.hedges@assembly.wales - Skipping (previously processed)
2025-02-01 12:28:46,488 - INFO - [8527/8512] mohammad.asghar@assembly.wales - Skipping (previously processed)
2025-02-01 12:28:46,488 - INFO - [8528/8512] neil.hamilton@assembly.wales - Skipping (previously processed)
2025-02-01 12:28:46,489 - INFO - [8529/8512] neil.mcevoy@assembly.wales - Skipping (previously processed)
2025-02-01 12:28:46,490 - INFO - [8530/8512] nicholas.ramsay@assembly.wales - Skipping (previously processed)
2025-02-01 12:28:46,490 - INFO - [8531/8512] paul.davies@assembly.wales - Skipping (previously processed)
2025-02-01 12:28:46,491 - INFO

Processing complete. Processed 15 emails with 0 errors.


### Get all the breaches

In [7]:
breach_url = "https://haveibeenpwned.com/api/v3/breaches"

response = requests.get(breach_url, headers=headers)
breaches_data = response.json()

In [8]:
breaches_df = pd.DataFrame(breaches_data)
breaches_df.head()

,Name,Title,Domain,BreachDate,AddedDate,ModifiedDate,PwnCount,Description,LogoPath,DataClasses,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree
0,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,"In approximately March 2015, the free web host...",https://haveibeenpwned.com/Content/Images/Pwne...,"[Email addresses, IP addresses, Names, Passwords]",True,False,False,False,False,False,False
1,123RF,123RF,123rf.com,2020-03-22,2020-11-15T00:59:50Z,2020-11-15T01:07:10Z,8661578,"In March 2020, the stock photo site <a href=""h...",https://haveibeenpwned.com/Content/Images/Pwne...,"[Email addresses, IP addresses, Names, Passwor...",True,False,False,False,False,False,False
2,126,126,126.com,2012-01-01,2016-10-08T07:46:05Z,2016-10-08T07:46:05Z,6414191,"In approximately 2012, it's alleged that the C...",https://haveibeenpwned.com/Content/Images/Pwne...,"[Email addresses, Passwords]",False,False,False,False,False,False,False
3,17Media,17,17app.co,2016-04-19,2016-07-08T01:55:03Z,2016-07-08T01:55:03Z,4009640,"In April 2016, customer data obtained from the...",https://haveibeenpwned.com/Content/Images/Pwne...,"[Device information, Email addresses, IP addre...",True,False,False,False,False,False,False
4,17173,17173,17173.com,2011-12-28,2018-04-28T04:53:15Z,2018-04-28T04:53:15Z,7485802,"In late 2011, <a href=""https://news.softpedia....",https://haveibeenpwned.com/Content/Images/Pwne...,"[Email addresses, Passwords, Usernames]",False,False,False,False,False,False,False


In [9]:
breaches_df.to_csv("../data/hipb_01_2025_breaches_data.csv", index=False)

### Get data for Bihar

From: https://vidhansabha.bih.nic.in/KnowyourMLA%20in%20Hindi.html

In [10]:
with open('../data/india/bihar/bihar.txt', 'r', encoding='utf-8') as file:
    lines = file.read().splitlines()
    lines = [line.split('\t') for line in lines]

bihar_df = pd.DataFrame(lines, columns=['sr_no', 'photo', 'constituency', 'name', 'gender', 'party', 'contact', 'email'])

In [11]:
output_folder = "../data/india/bihar_hibp/"
processor = HIBPProcessor(output_folder, headers, payload, 10)
bihar_df = clean_email_column(bihar_df)
stats = processor.process_dataframe(bihar_df)
print(f"Processing complete. Processed {stats['processed']} emails with {stats['errors']} errors.")

2025-02-01 12:28:46,787 - INFO - Starting to process 241 emails
2025-02-01 12:28:46,788 - INFO - [1/241] mla-vnagar-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,788 - INFO - [2/241] mla-ramnagar-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,789 - INFO - [3/241] mla-nktganj-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,789 - INFO - [4/241] mla-bagaha-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,789 - INFO - [5/241] mla-lauriya-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,790 - INFO - [6/241] mla-nautan-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,790 - INFO - [7/241] mla-chanpatiya-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,791 - INFO - [8/241] mla-bettiah-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,791 - INFO - [9/241] mla-sikata-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,791 - INFO - [10/241] mla-raxaul-b

2025-02-01 12:28:46,825 - INFO - [85/241] mla-bahadurpur-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,825 - INFO - [86/241] mla-kevati-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,826 - INFO - [87/241] mla-jale-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,826 - INFO - [88/241] mla-gaighat-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,827 - INFO - [89/241] mla-aurai-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,827 - INFO - [90/241] mla-minapur-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,828 - INFO - [91/241] mla-bochaha-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,828 - INFO - [92/241] mla-sakara-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,828 - INFO - [93/241] mla-kudhani-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,829 - INFO - [94/241] mla-muz-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:4

2025-02-01 12:28:46,862 - INFO - [168/241] mla-lakhisarai-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,862 - INFO - [169/241] mla-sheikhpura-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,863 - INFO - [170/241] mla-barbigha-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,863 - INFO - [171/241] mla-asthawa-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,864 - INFO - [172/241] mla-biharsharif-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,864 - INFO - [173/241] mla-rajgir-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,865 - INFO - [174/241] mla-islampur-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,865 - INFO - [175/241] mla-hilsa-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,865 - INFO - [176/241] mla-nalanda-bih@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,866 - INFO - [177/241] mla-harnaut-bih@nic.in - Skipping (previously pr

Processing complete. Processed 0 emails with 0 errors.


### HP

In [12]:
hp_df = pd.read_csv("../data/india/hp_14.csv")
hp_df.columns = hp_df.columns.str.lower()

In [13]:
output_folder = "../data/india/hp_hibp/"
hp_df = clean_email_column(hp_df)
processor = HIBPProcessor(output_folder, headers, payload, 10)
stats = processor.process_dataframe(hp_df)
print(f"Processing complete. Processed {stats['processed']} emails with {stats['errors']} errors.")

2025-02-01 12:28:46,919 - INFO - Starting to process 68 emails
2025-02-01 12:28:46,920 - INFO - [1/68] solankeyajay20@gmail.com - Skipping (previously processed)
2025-02-01 12:28:46,921 - INFO - [2/68] anil.kumar15@nic.in - Skipping (previously processed)
2025-02-01 12:28:46,921 - INFO - [3/68] anirudhsingh@iyc.in - Skipping (previously processed)
2025-02-01 12:28:46,922 - INFO - [4/68] ashbut209@gmail.com - Skipping (previously processed)
2025-02-01 12:28:46,922 - INFO - [5/68] ashish.sharma.mla@gmail.com - Skipping (previously processed)
2025-02-01 12:28:46,922 - INFO - [6/68] verma.balbir341@gmail.com - Skipping (previously processed)
2025-02-01 12:28:46,923 - INFO - [7/68] bhawanipathania@gmail.com - Skipping (previously processed)
2025-02-01 12:28:46,923 - INFO - [8/68] gaurbhuvneshwar1974@gmail.com - Skipping (previously processed)
2025-02-01 12:28:46,924 - INFO - [9/68] mlabikramthakur64@gmail.com - Skipping (previously processed)
2025-02-01 12:28:46,924 - INFO - [10/68] mlagagr

Processing complete. Processed 0 emails with 0 errors.


### TN

From https://assembly.tn.gov.in/16thassembly/members.php

In [14]:
tn_df = pd.read_csv("../data/india/tn/tn.csv")
tn_df.rename(columns={"Email Address": "email"}, inplace=True)

In [15]:
output_folder = "../data/india/tn_hibp/"
processor = HIBPProcessor(output_folder, headers, payload, 10)
tn_df = clean_email_column(tn_df)
stats = processor.process_dataframe(tn_df)
print(f"Processing complete. Processed {stats['processed']} emails with {stats['errors']} errors.")

2025-02-01 12:28:46,965 - INFO - Starting to process 233 emails
2025-02-01 12:28:46,966 - INFO - [1/233] mlavelachery@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:46,966 - INFO - [2/233] mlamanapparai@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:46,967 - INFO - [3/233] mlapalayamkottai@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:46,967 - INFO - [4/233] mlapolur@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:46,968 - INFO - [5/233] mlavandavasi@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:46,968 - INFO - [6/233] mlacoimbatorenorth@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:46,969 - INFO - [7/233] mlavalparai@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:46,969 - INFO - [8/233] mlagudiyattam@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:46,970 - INFO - [9/233] mlapalladam@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:46,970 - INFO - [10/233] mlakum

2025-02-01 12:28:47,003 - INFO - [84/233] mlathiyagarayanagar@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,003 - INFO - [85/233] mlabhavani@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,004 - INFO - [86/233] mlaramanathapuram@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,004 - INFO - [87/233] mlamanachanallur@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,005 - INFO - [88/233] mladharapuram@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,005 - INFO - [89/233] mlakadayanallur@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,006 - INFO - [90/233] mlapoonamallee@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,006 - INFO - [91/233] mlaviluppuram@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,007 - INFO - [92/233] mlaandipatti@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,007 - INFO - [93/233] mlamadathukulam@tn.gov.in - Skipping (previous

2025-02-01 12:28:47,038 - INFO - [165/233] mlathalli@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,039 - INFO - [166/233] mlaaranthangi@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,039 - INFO - [167/233] mlanamakkal@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,039 - INFO - [168/233] mlaarakkonam@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,040 - INFO - [169/233] mlathirumayam@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,040 - INFO - [170/233] mlananguneri@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,041 - INFO - [171/233] mlavasudevanallur@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,041 - INFO - [172/233] mlamettur@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,042 - INFO - [173/233] mlaoddanchatram@tn.gov.in - Skipping (previously processed)
2025-02-01 12:28:47,042 - INFO - [174/233] mlakangayam@tn.gov.in - Skipping (previously processed

Processing complete. Processed 0 emails with 0 errors.


### UP

In [16]:
up_df = pd.read_csv("../data/india/up/up_18_mlas.csv", usecols=['email'])

In [17]:
output_folder = "../data/india/up_hibp/"
up_df = clean_email_column(up_df)
processor = HIBPProcessor(output_folder, headers, payload, 10)
stats = processor.process_dataframe(up_df)
print(f"Processing complete. Processed {stats['processed']} emails with {stats['errors']} errors.")

2025-02-01 12:28:47,094 - INFO - Starting to process 106 emails
2025-02-01 12:28:47,095 - INFO - [1/106] umarkhanoffice@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,095 - INFO - [2/106] mukeshradi@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,095 - INFO - [4/106] ashumalik.samajwadiparty@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,096 - INFO - [5/106] singhbrijesh2003@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,096 - INFO - [10/106] chaudharyprasanna11@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,097 - INFO - [11/106] rajpalsinghbaliyan456@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,097 - INFO - [13/106] anilkm013@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,098 - INFO - [16/106] narayanniwas188@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,098 - INFO - [22/106] suchimausammla@gmail.com - Skipping (previously processed)
2025-02-01 12:2

2025-02-01 12:28:47,131 - INFO - [313/106] ganeshchandskn@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,131 - INFO - [315/106] rishibjp1983@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,132 - INFO - [316/106] premsagarpatal62@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,132 - INFO - [323/106] pradeepshukla.bjp@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,132 - INFO - [327/106] rajeshtripati.mla@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,133 - INFO - [328/106] vivekanandbjp@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,133 - INFO - [333/106] m9794077973@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,134 - INFO - [336/106] shalabh.pat@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,135 - INFO - [339/106] sabhakunwark@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,135 - INFO - [340/106] vijaylaxmigautam.72@gmail.com - Skipping (

Processing complete. Processed 0 emails with 0 errors.


### Delhi

In [18]:
del_df = pd.read_csv("../data/india/delhi/delhi_7th_assembly.csv", usecols=['Email'])
del_df.rename(columns={"Email": "email"}, inplace=True)
del_df = clean_email_column(del_df)

In [19]:
output_folder = "../data/india/delhi_hibp/"
processor = HIBPProcessor(output_folder, headers, payload, 10)
stats = processor.process_dataframe(del_df)
print(f"Processing complete. Processed {stats['processed']} emails with {stats['errors']} errors.")

2025-02-01 12:28:47,164 - INFO - Starting to process 43 emails
2025-02-01 12:28:47,165 - INFO - [1/43] akhilesht84@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,166 - INFO - [2/43] aapkasomdutt@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,166 - INFO - [3/43] parladsinghsawhney@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,167 - INFO - [4/43] shoaib.iqbal2@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,167 - INFO - [5/43] imran.hussain@gov.in - Skipping (previously processed)
2025-02-01 12:28:47,168 - INFO - [6/43] delhidvs@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,168 - INFO - [8/43] s.c.6262@gmail.com - Skipping (previously processed)
2025-02-01 12:28:47,169 - INFO - [9/43] mascotleathercraft@yahoo.com - Skipping (previously processed)
2025-02-01 12:28:47,325 - WARNING - Rate limit hit. Waiting 6 seconds
2025-02-01 12:28:55,331 - INFO - [11/43] rajkumaridhillon@gmail.com - Skipping (previousl

Processing complete. Processed 0 emails with 1 errors.


### Norway

In [21]:
no_parl = pd.read_csv("../data/no/no_parliament.csv")
output_folder = "../data/no_hibp/"
no_parl = clean_email_column(no_parl)
processor = HIBPProcessor(output_folder, headers, payload, 10)
stats = processor.process_dataframe(no_parl)
print(f"Processing complete. Processed {stats['processed']} emails with {stats['errors']} errors.")

2025-02-01 12:30:57,671 - INFO - Starting to process 174 emails
2025-02-01 12:30:57,672 - INFO - [1/174] anja.ninasdotter.abusland@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,673 - INFO - [2/174] grunde.kreken.almeland@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,673 - INFO - [3/174] per-willy.amundsen@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,674 - INFO - [4/174] marit.arnstad@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,675 - INFO - [5/174] henrik.asheim@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,676 - INFO - [6/174] jorodd.asphjell@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,677 - INFO - [7/174] nikolai.astrup@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,678 - INFO - [8/174] asmund.aukrust@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,679 - INFO - [9/174] seher.aydar@stortinget.no - Skipping (pre

2025-02-01 12:30:57,719 - INFO - [78/174] kathrine.kleveland@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,720 - INFO - [79/174] jenny.klinge@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,720 - INFO - [80/174] lisa.marie.ness.klungland@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,722 - INFO - [81/174] eigil.knutsen@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,723 - INFO - [82/174] turid.kristensen@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,724 - INFO - [83/174] mimir.kristjansson@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,725 - INFO - [84/174] erlend.larsen@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,725 - INFO - [85/174] stein-erik.lauvas@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,725 - INFO - [86/174] kirsti.leirtro@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,726 - INFO - [8

2025-02-01 12:30:57,776 - INFO - [154/174] hadia.tajik@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,776 - INFO - [155/174] bard.ludvig.thorheim@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,777 - INFO - [156/174] ingvild.wetrhus.thorsvik@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,777 - INFO - [157/174] ove.trellevik@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,778 - INFO - [158/174] anette.trettebergstuen@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,778 - INFO - [159/174] tone-wilhelmsen.troen@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,778 - INFO - [160/174] christian.tybring-gjedde@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,779 - INFO - [161/174] mathilde.tybring-gjedde@stortinget.no - Skipping (previously processed)
2025-02-01 12:30:57,779 - INFO - [162/174] per.olav.tyldum@stortinget.no - Skipping (previously processed)

Processing complete. Processed 0 emails with 0 errors.


### Denmark

In [22]:
wb = openpyxl.load_workbook("../data/danish_parliament_1_2025.xlsx")
ws = wb.active  
data = list(ws.values)
dk_parl = pd.DataFrame(data)
dk_parl.columns = dk_parl.iloc[0]
dk_parl = dk_parl[1:].reset_index(drop=True)

/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [23]:
output_folder = "../data/dk_hibp/"
dk_parl.rename(columns={"Email": "email"}, inplace=True)
dk_parl = clean_email_column(dk_parl)
processor = HIBPProcessor(output_folder, headers, payload, 10)
stats = processor.process_dataframe(dk_parl)
print(f"Processing complete. Processed {stats['processed']} emails with {stats['errors']} errors.")

2025-02-01 12:31:02,998 - INFO - Starting to process 186 emails
2025-02-01 12:31:02,999 - INFO - [1/186] akimatilda@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,000 - INFO - [2/186] alex.ahrendtsen@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,001 - INFO - [3/186] alex.vanopslagh@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,003 - INFO - [4/186] alexander.ryle@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,004 - INFO - [5/186] anders.kronborg@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,005 - INFO - [6/186] ministeren@bm.dk - Skipping (previously processed)
2025-02-01 12:31:03,007 - INFO - [7/186] anna.falkenberg@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,008 - INFO - [8/186] anne.paulin@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,010 - INFO - [9/186] anne.berthelsen@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,011 - INFO - [10/186] anni.matthiesen@ft.dk - Skipping (pr

2025-02-01 12:31:03,062 - INFO - [85/186] lasse.pedersen@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,062 - INFO - [86/186] lea.wermelin@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,063 - INFO - [87/186] leif.jensen@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,063 - INFO - [88/186] leila.stockmarr@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,064 - INFO - [89/186] linea@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,064 - INFO - [90/186] lisbeth.bech-nielsen@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,064 - INFO - [91/186] lise.bech@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,065 - INFO - [92/186] lise.bertelsen@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,065 - INFO - [93/186] lotte.rod@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,066 - INFO - [94/186] louise.brown@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,066 - INFO - [95/186] louise.

2025-02-01 12:31:03,099 - INFO - [170/186] tanja.larsson@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,099 - INFO - [171/186] theresa.scavenius@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,099 - INFO - [172/186] theresa.berg.andersen@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,100 - INFO - [173/186] trm@trm.dk - Skipping (previously processed)
2025-02-01 12:31:03,100 - INFO - [174/186] thomas.jensen@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,100 - INFO - [175/186] thomas.monberg@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,101 - INFO - [176/186] thomas.skriver@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,101 - INFO - [177/186] thorkild.holmboe-hay@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,101 - INFO - [178/186] tobias.elmstroem@ft.dk - Skipping (previously processed)
2025-02-01 12:31:03,102 - INFO - [179/186] torsten.gejl@ft.dk - Skipping (previously processed)
2025-02-01 12:31

Processing complete. Processed 0 emails with 0 errors.


### India LS

In [ ]:
in_parl = pd.read_csv("../data/india/ls.csv")

output_folder = "../data/india/ls_hibp/"
in_parl = clean_email_column(in_parl)
processor = HIBPProcessor(output_folder, headers, payload, 10)
stats = processor.process_dataframe(in_parl)
print(f"Processing complete. Processed {stats['processed']} emails with {stats['errors']} errors.")